In [145]:
import pandas as pd

data = pd.read_csv('../data/diseases.csv')
print(len(data))

696


## Investigate data

In [146]:
risk_factor = data.loc[data['risk_factor'].notnull()]['risk_factor'].tolist()
symptom_data = data.loc[data['symptom'].notnull()]['symptom'].tolist()
# risk_factor.count() # 65 non-Nan value

In [147]:
risk_factor[:3]

['Thiểu năng tuần hoàn não do các nguyên nhân như xơ vữa động mạch (gây ra do huyết áp cao, mỡ máu cao, đái tháo đường, hút thuốc lá), bệnh tim (hẹp van tim, suy tim), huyết áp thấp. \n\nBệnh thường gặp ở những người trung niên, đặc biệt những người lao động trí óc. Bệnh ngày càng có xu hướng trẻ hóa do áp lực lớn từ công việc, cuộc sống cùng với thói quen ăn uống sinh hoạt không lành mạnh như lạm dụng bia, rượu, thuốc lá...',
 'Bất kỳ ai cũng có thể bị nhiễm HIV/AIDS nếu nằm trong đường lây truyền. Những người có nguy cơ cao bị mắc bệnh AIDS như:\n\n- Quan hệ tình dục không an toàn, có nghĩa là giao hợp âm đạo hoặc hậu môn hoặc giao hợp bằng miệng mà không dùng bao cao su với một người bị nhiễm HIV;\n- Sử dụng chung kim tiêm để tiêm thuốc hoặc steroid với người bị nhiễm bệnh. Ngoài ra, bệnh cũng có thể lây qua kim tiêm bẩn được sử dụng để xăm hình hoặc xỏ lỗ trên cơ thể;\n- Bị truyền máu từ một người nhiễm bệnh HIV/AIDS.\n- Một em bé cũng có thể bị nhiễm HIV từ sữa mẹ nếu người phụ nữ

In [203]:
import string
import re
from typing import List, Dict

PATTERNS = [
    "\*\*(.*?)\*\*",
    "(.*?)(\,|\.|\()"
]

EXTEND_ENTITES = [
    "thanh thiếu niên",
    "mọi lứa tuổi",
    "thiếu niên",
    "thanh niên",
    "trung niên",
    "lao động trí óc",
    "huyết áp thấp",
    "lao động chân tay",
    "thừa cân",
    "trẻ em",
    "nam",
    "nữ",
    "tuổi",
    "mang thai"
]

class NER:
    ''' Entity Extraction module
    '''
    def __init__(self):
        self.patterns = PATTERNS 
        self.ex_entities = EXTEND_ENTITES
        
    @staticmethod
    def is_format_1(sent:str) -> bool:
        ''' Format has ":" at the end of the sentence
        '''
        if 'như:' in sent and text.find('như') > len(sent)//2:
            return True
        if 'gồm:' in sent and text.find('gồm') > len(sent)//2:
            return True
        if ':' in sent:
            return True
        return False
    
    @staticmethod
    def is_format_2(sent:str) -> bool:
        ''' Format has "-" at the beginning of the sentnece
        '''
        if '-' in sent and sent.find('-') in list(range(0,5)):
            return True
        return False
    
    @staticmethod
    def is_format_3(sent:str) -> bool:
        ''' Format has "gồm có:" ...,....
        '''
        if "gồm có:" in sent:
            return True
        return False
    
    def is_format_4(self,sent:str) -> bool:
        ''' Contains needed keywords
        '''
        for entity in self.ex_entities:
            if sent.find(entity) != -1 or sent.find(entity.lower()) != -1:
                return True
        return False
    
    def get_entites(self,text:str,method='rule') -> List:
        assert method in ['rule','deep']
        if method == 'rule':
            entities = self.get_entities_rule_base(text)
            
        if method == 'deep':
            pass
        
        return entities

    def get_entities_rule_base(self,sentences:str) -> List[str]:
        ''' Return list of entities
        '''
        sentences = sentences.split('\n')
        # get potential sentences
        potential_sentences = self.get_potential_sentences(sentences)
        
        # extract entities from potential sentences
        entites = self.get_entites_from_potential_sentences(potential_sentences)
        
        # Normalizing entites
        entites = self.post_normalize_entities(entites)
        
        return entites
    
    def post_normalize_entities(self,entities) -> List[str]:
        result = []
        for ent in entities:
            e = ent.translate(str.maketrans('', '', string.punctuation)).strip()
            result.append(e)
        return result

    def get_entites_from_potential_sentences(self,pot_sentences:List[str]) -> List[str]:
        ''' Return list of entities from potential sentences
        '''
        entities = []
        for sent in pot_sentences:
            is_selected = False
            for pattern in self.patterns:
                # has been accpeted by other patterns
                if is_selected:
                    break
                try:
                    e = re.search(pattern, sent).group(1)
                    entities.append(e)
                    is_selected=True
                except:
                    continue
                    
            # still not extract anything. Use dict-based
            if is_selected == False:
                e = self.get_entities_dict_based(sent) 
                entities.extend(e)
            else:
                # update self.ex_entities
                for e in entities:
                    if e not in self.ex_entities:
                        self.ex_entities.append(e)
        
        return entities
    
    def get_entities_dict_based(self,sent:str) -> List[str]:
        result = []
        for entity in self.ex_entities:
            if sent.find(entity) != -1 or sent.find(entity.lower()) != -1:
                result.append(entity)
        return result
    
    def get_potential_sentences(self,sentences:str) -> List[str]:
        ''' Return list of sentences that potentially contain risk_factor entities
        '''
        result = []
        is_first_format = False

        for sent in sentences:
            if sent == '':
                continue
                
            if is_first_format:
                result.append(sent)
            elif self.is_format_2(sent):
                result.append(sent)
            elif self.is_format_3(sent):
                result.append(sent)
            elif self.is_format_4(sent):
                result.append(sent)
                
            if self.is_format_1(sent):
                is_first_format = True
                continue
                
        return result
   

In [204]:
ner = NER()
text = risk_factor[2]
entites = ner.get_entites(text,method='rule')
print(entites)

['Trẻ sơ sinh', 'Trẻ em', 'Người lớn 65 tuổi', 'Phụ nữ mang thai hoặc đang cho con bú']


In [205]:
rf_entites = []
not_cover_cnt = 0
failure_sentences = []

for index, sent in enumerate(risk_factor):
    entites = ner.get_entites(sent,method='rule')
    if entites != []:
        rf_entites.extend(entites)
    else:
        not_cover_cnt += 1
        failure_sentences.append(sent)
print(len(rf_entites))
print(f"# cover cases: {len(risk_factor) - not_cover_cnt} / {len(risk_factor)}")

271
# cover cases: 64 / 65


In [206]:
rf_entites[:10]

['Thiểu năng tuần hoàn não do các nguyên nhân như xơ vữa động mạch',
 'Bệnh thường gặp ở những người trung niên',
 'Quan hệ tình dục không an toàn',
 'Sử dụng chung kim tiêm để tiêm thuốc hoặc steroid với người bị nhiễm bệnh',
 'Bị truyền máu từ một người nhiễm bệnh HIVAIDS',
 'Một em bé cũng có thể bị nhiễm HIV từ sữa mẹ nếu người phụ nữ nhiễm bệnh',
 'Trẻ sơ sinh',
 'Trẻ em',
 'Người lớn 65 tuổi',
 'Phụ nữ mang thai hoặc đang cho con bú']

In [207]:
entites_length = [len(item.split(' ')) for item in rf_entites]
print(sum(entites_length)/len(entites_length))

9.822878228782288


In [208]:
64/65

0.9846153846153847

### Investigate failure cases

In [209]:
failure_sentences[:5]

['Đối tượng  có nguy cơ mắc bệnh phong cao nhất là những người sống ở khu vực có nhiều người mắc bệnh phong, có thể kể đến một số khu vực như một số địa phận thuộc Ấn Độ, Trung Quốc, Nhật Bản, Nepal, Ai Cập,...  đặc biệt là những người tiếp xúc thường xuyên với người nhiễm bệnh phong.\n\nMột số bệnh nhân có các khuyết tật di truyền trong hệ thống miễn dịch như là khuyết tật ở vùng Q25 trên nhiễm sắc thể 6 có khả năng năng cao bị nhiễm bệnh lý này\n\nNhững người thường xuyên xử lý một số động vật nhất định như armadillos, tinh tinh châu Phi, khỉ mặt xanh cổ trắng và khỉ đuôi dài, được biết có mang theo vi khuẩn phong do bị lây nhiễm vi khuẩn từ động vật, đặc biệt nếu họ không đeo găng tay bảo vệ khi xử lý các động vật này.']

### Extend entities dict 

In [155]:
EXTEND_ENTITES = [
    "thanh thiếu niên",
    "mọi lứa tuổi",
    "thiếu niên",
    "thanh niên",
    "trung niên",
    "lao động trí óc",
    "huyết áp thấp",
    "lao động chân tay",
    "thừa cân"
]
rf_entites.extend(EXTEND_ENTITES)
print(len(rf_entites))

209


In [156]:
cnt = 0
tmp_ent = []
ff_sent = []
for text in failure_sentences:
    is_found = False
    for sent in text.split('\n'):
        for entity in rf_entites:
            if sent.find(entity) != -1 or sent.find(entity.lower()) != -1:
                is_found = True
#                 print({sentence} --> {entity})
                tmp_ent.append(entity)
    if is_found:
        cnt += 1
    else:
        ff_sent.append(text)

In [157]:
tmp_ent = list(set(tmp_ent))
print(len(tmp_ent), len(failure_sentences))
print(tmp_ent[:10])

16 16
['thiếu niên', 'Ăn uống', 'lao động chân tay', 'Thanh niên', 'Trẻ em', 'thừa cân', 'thanh niên', 'trung niên', 'huyết áp thấp', 'lao động trí óc']


### Entity Resolution

Suported risk factor:
- Age (trẻ em,thiếu niên,thanh niên,trung niên,cao tuổi)
- Pregnant status (có/không)
- Family History (todo:hard)
- Personal History (todo:hard)

In [95]:
import copy

rf_entities_copy = copy.deepcopy(rf_entites)
rf_entities_copy = list(set(rf_entities))
print(len(rf_entities_copy))

206


In [242]:
from typing import Dict

KEYWORDS = {
    'age': ["trẻ em","thanh niên","trung niên","cao tuổi","tuổi","niên"],
    'pregnant':["mang thai","nữ"],
    'family_history':["gia đình"],
    'personal_history':["tiền sử"]
}
RESOLUTION_ENTITIES = {
    "age" : {
        "trẻ em" : ["trẻ em"], # 0 - 10
        "thiếu niên" : ["thiếu niên"], # 11 - 18
        "thanh niên" : ["thanh niên"], # 18 - 40
        "trung niên" : ["trung niên"], # 40-65
        "cao tuổi": ["cao tuổi","tuổi cao", "65 tuổi"], # > 65
        "mọi lứa tuổi": ["mọi lứa tuổi",'tuổi']
    },
    "pregnant" : {
        "phụ nữ mang thai" : ["mang thai"],
    }
}
class EntityResolution:
    def __init__(self):
        self.keywords = KEYWORDS
        self.resolution_entites = RESOLUTION_ENTITIES
    
    def __call__(self,entities) -> List[Dict]:
        ''' Return new list of dict entities has align to self.resolution_entites
        '''    
        result = []
        
        if type(entities) == list:
            for ent in entities:
                tmp = {}
                for entity_type in list(self.resolution_entites.keys()):
                    # Check whether given entity contains keyword
                     if self.is_contains_entities(ent,entity_type=entity_type):
                        
                        # Get resolution entities align to entity_type
                        united_entites = self.get_resolution_entities(ent,entity_type=entity_type)
                        if united_entites != {}:
                            tmp[entity_type] = united_entites
                result.append(tmp)
        else:
            for entity_type in list(self.resolution_entites.keys()):
                # Get resolution entities align to entity_type
                united_entites = self.get_resolution_entities(entities,entity_type=entity_type)
                if united_entites != {}:
                    result[entity_type] = united_entites
        
        # filter empty {}
        result = [item for item in result if item != {}]
        
        return result
                
    @staticmethod
    def normalize_entity(entity) -> str:
        entity = entity.lower()
        return entity
    
    
    def is_contains_entities(self,entity:str,entity_type:str='age') -> bool:
        assert entity_type in list(self.keywords.keys())
        
        keywords = self.keywords[entity_type] # List
        
        for prf in keywords:
            if entity.lower().find(prf) != -1:
                return True
        
        return False
    
    def get_resolution_entities(self,entity:str,entity_type:str='age') -> List[str]:
        assert entity_type in list(self.keywords.keys())

        result = []
        resolution_entities = self.resolution_entites[entity_type] # dict
        
        for k, v in resolution_entities.items():
            for synonym in v:
                if entity.lower().find(synonym) != -1:
                    result.append(k)
                    break
        
        return result
    
    def save(self) -> None:
        ''' Save resolution entities into csv file align to input
        '''
        pass
        # TODO
        
    

In [243]:
entity_resolution = EntityResolution()
resolution_rf_entities = entity_resolution(rf_entities)

In [244]:
''' DATA
'''
import pandas as pd

data = pd.read_csv('../data/diseases.csv')

risk_factor = data.loc[data['risk_factor'].notnull()]['risk_factor'].tolist()

In [275]:
risk_factor = data['risk_factor'].tolist()
risk_factor = [item if type(item) == str else '0' for item in risk_factor]
print(risk_factor[:5])

['0', '0', '0', '0', '0']


In [287]:
''' NER + ENTITY RESOLUTION
'''
import math

risk_factor = data['risk_factor'].tolist()
risk_factor = [item if type(item) == str else '0' for item in risk_factor]

ner = NER()
entity_resolution = EntityResolution()


rf_entites = []
not_cover_cnt = 0
failure_sentences = []

age_cnt = 0
pregnant_cnt = 0

for index, sent in enumerate(risk_factor):
    if sent == '0':
        rf_entites.append([])
    else:
        entites = ner.get_entites(sent,method='rule')
        if entites != []:
            entites = entity_resolution(entites)
            if entites != []:
                rf_entites.append(entites)
                if 'age' in list(entites[0].keys()):
                    age_cnt += 1
                if 'pregnant' in list(entites[0].keys()):
                    pregnant_cnt += 1
            else:
                rf_entites.append([])
        else:
            not_cover_cnt += 1
            failure_sentences.append(sent)
            rf_entites.append([])
# print(len(rf_entites))
# print(f"# cover cases: {len(risk_factor) - not_cover_cnt} / {len(risk_factor)}")

In [282]:
print(len(risk_factor),len(rf_entites))

696 696


In [288]:
pregnant_cnt, age_cnt

(8, 32)

In [283]:
data['risk_factor_entity'] = rf_entites
data.to_csv('../data/diseases_rf_entity.csv')

### Analysis/Debug of entity resolution

In [118]:
'''
Something that increases the chance of developing a disease
age : ["trẻ em","thanh thiếu niên","trung niên"]
a family history of certain cancers, 
use of tobacco products, 
being exposed to radiation or certain chemicals, 
infection with certain viruses or bacteria, 
and certain genetic changes.
'''

rf_entities_copy = copy.deepcopy(rf_entites)
rf_entities_copy = list(set(rf_entities))


# trẻ em: Dưới 16 tuổi. Vị thành niên: Từ đủ 16 tuổi đến dưới 18 tuổi. Thành niên: Từ đủ 18 tuổi trở lên.
AGE_RF = [
    "trẻ em", # 0 - 10
    "thiếu niên", # 11 - 18
    "thanh niên" # 18 - 40
    "trung niên",# 40-65
    "cao tuổi", # > 65
    "tuổi",
    "niên"
]

age_entities = []
fh_entities = []
ph_entities = []
pregnant_entities = []

# AGE
for ent in rf_entities_copy:
    for prf in AGE_RF:
        if ent.lower().find(prf) != -1:
            age_entities.append(ent)
            try:
                rf_entities_copy.remove(ent)
            except:
                pass
            break

# FAMILY HISTORY
FH_RF = [
    "gia đình",
]
for ent in rf_entites:
    for prf in FH_RF:
        if ent.lower().find(prf) != -1:
            fh_entities.append(ent)
            try:
                rf_entities_copy.remove(ent)
            except:
                pass
            break
            
# PERSONAL HISTORY
PH_RF = [
    "tiền sử"
]
for ent in rf_entites:
    for prf in PH_RF:
        if ent.lower().find(prf) != -1:
            ph_entities.append(ent)
            try:
                rf_entities_copy.remove(ent)
            except:
                pass
            break

# SEX
PREGNANT_RF = [
    "mang thai",
    "nữ"
]
for ent in rf_entites:
    for prf in PREGNANT_RF:
        if ent.lower().find(prf) != -1:
            pregnant_entities.append(ent)
            try:
                rf_entities_copy.remove(ent)
            except:
                pass
            break

In [119]:
print(len(age_entities),len(fh_entities),len(ph_entities),len(sex_entities))
print(len(rf_entities_copy)) # filtered

21 9 11 12
158


In [110]:
AGE_RF = {
    "trẻ em" : ["trẻ em"], # 0 - 10
    "thiếu niên" : ["thiếu niên"], # 11 - 18
    "thanh niên" : ["thanh niên"], # 18 - 40
    "trung niên" : ["trung niên"], # 40-65
    "cao tuổi": ["cao tuổi","tuổi cao", "65 tuổi"], # > 65
    "mọi lứa tuổi": ["mọi lứa tuổi",'tuổi']
}

cnt = 0
a = []
for ent in age_entities:
    tmp = []
    for age, v in AGE_RF.items():
        for synonym in v:
            if ent.lower().find(synonym) != -1:
                tmp.append(age)
                break
    a.append(tmp)
    if tmp == []:
        print(ent)
print(a) # If print --> not cover 100%

[['trẻ em'], ['mọi lứa tuổi'], ['trẻ em', 'mọi lứa tuổi'], ['trẻ em', 'mọi lứa tuổi'], ['mọi lứa tuổi'], ['cao tuổi', 'mọi lứa tuổi'], ['trẻ em', 'mọi lứa tuổi'], ['cao tuổi', 'mọi lứa tuổi'], ['trẻ em', 'mọi lứa tuổi'], ['trẻ em', 'cao tuổi', 'mọi lứa tuổi'], ['thiếu niên'], ['mọi lứa tuổi'], ['mọi lứa tuổi'], ['thiếu niên'], ['mọi lứa tuổi'], ['thanh niên'], ['mọi lứa tuổi'], ['cao tuổi', 'mọi lứa tuổi'], ['trung niên'], ['cao tuổi', 'mọi lứa tuổi'], ['thanh niên']]


In [120]:
ph_entities

['Những bệnh nhân có tiền sử bị thủy đậu',
 'Tiền sử bị chóng mặt',
 'Gia đình có người từng có tiền sử mắc bệnh Alzheimer',
 'Có tiền sử hoặc đang mắc các bệnh về gan',
 'Tiền sử đột quỵ cũ Bệnh nhân có tiền sử đột quỵ cũ có nhiều nguy cơ tái phát',
 'Có tiền sử mắc các bệnh lý về tuyến giáp như nhiễm trùng',
 'Dị ứng và hen Những người có tiền sử gia đình hoặc cá nhân bị bệnh chàm',
 'Người có tiền sử viêm nhiễm các cơ quan lân cận với xoang Viêm mũi',
 'Người có tiền sử gia đình có người bị sỏi thận',
 'Tiền sử đã từng ăn cá sống ở vùng dịch tễ',
 'Người có tiền sử mắc các bệnh viêm nhiễm đại tràng mãn tính như viêm loét đại tràng chảy máu']

In [ ]:
FH_RF = [
    "gia đình",
    "tiền sử"
]

cnt = 0
for ent in rf_entites:
    for prf in FH_RF:
        if ent.lower().find(prf) != -1:
            print(ent)
            cnt += 1
            break
print(cnt)